In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Import all the necessary libraries, paths, and constants

In [2]:
import numpy as np
import tensorflow as tf

Adjacency matrix

In [3]:
A = np.array([[0, 1, 1, 1, 1, 0, 0, 0, 0],[1, 0, 1, 1, 1, 0, 0, 0, 0],[1, 1, 0, 1, 1, 0, 0, 0, 0],[1, 1, 1, 0, 1, 1, 0, 0, 0],[1,1,1,1,0,0,0,0,0],[0,0,0,1,0,0,1,1,1],[0,0,0,0,0,1,0,1,1],[0,0,0,0,0,1,1,0,1],[0,0,0,0,0,1,1,1,0]])

Feature Matrix

In [4]:
F1 = np.array([[1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,1]])

Degree Matrix

In [5]:
D =np.array([[4,0,0,0,0,0,0,0,0],[0,4,0,0,0,0,0,0,0],[0,0,4,0,0,0,0,0,0],[0,0,0,5,0,0,0,0,0],[0,0,0,0,4,0,0,0,0],[0,0,0,0,0,4,0,0,0],[0,0,0,0,0,0,3,0,0],[0,0,0,0,0,0,0,3,0],[0,0,0,0,0,0,0,0,3]])

Adjacency matrix with self connection

In [6]:
A_hat =A + F1

Labels

In [7]:
labels =[[1,0],[1,0],[1,0],[1,0],[1,0],[0,1],[0,1],[0,1],[0,1]]

Length of each label

In [8]:
ldim = len(labels[0])

Node Indices

In [9]:
train_index = []
for i in range(len(A)):
    train_index.append(i)

GCN implementation

In [10]:
def gnn(adj_matrix, degree_matrix,feature_matrix,label,train_index):
    features = tf.placeholder(tf.float32, shape = ((None,len(feature_matrix))))
    adjacency = tf.placeholder(tf.float32, shape = ((None,None)))
    degree = tf.placeholder(tf.float32, shape = ((None,None)))
    labels = tf.placeholder(tf.float32, shape = ((None,ldim)))
    weights1 = tf.Variable(tf.random_normal([len(feature_matrix),512], stddev = 1))
    weights2 = tf.Variable(tf.random_normal([512,ldim], stddev = 1))
    trainIndex = tf.placeholder(tf.int32, shape = ((len(train_index))))
#Defining GCN layer
    def layer(features, adjacency, degree, weights):
        with tf.name_scope('gcn_layer'):
            d_ = tf.pow(tf.matrix_inverse(degree), 0.5)
            y = tf.matmul(d_, tf.matmul(adjacency, d_))
            kernel = tf.matmul(features, weights)
            return tf.nn.relu(tf.matmul(y, kernel))

#Building with GCN layer        
    hidden1 = layer(features, adjacency, degree, weights1)
    hidden1 = tf.layers.dropout(hidden1, rate=0.5)
    model = layer(hidden1, adjacency, degree, weights2)
    training_output = tf.gather(model, trainIndex)

#Defining loss function and optimizer
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = training_output, labels = label.astype(np.float32)))
        train_op = tf.train.AdamOptimizer(0.01, 0.90).minimize(loss)
#creating session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

#training up to 200 epochs
    for i in range(200):
        _, cost = sess.run([train_op, loss], feed_dict = {features: feature_matrix, adjacency: adj_matrix, degree: degree_matrix, labels: label, trainIndex: train_index})
        if(i%1 == 0):
            predict = sess.run(tf.cast(tf.nn.softmax(model)>0.4, tf.int32,name='predictions'), feed_dict = {features: feature_matrix, adjacency: adj_matrix, degree: degree_matrix,labels: label})
            print(predict)
        print("epoch:",i) 
    return predict,label, training_output

Train and Predict

In [11]:
test_res,test_label,t= gnn(A_hat, D, F1, np.array(labels),train_index)

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 0
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 1
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 2
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 3
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 4
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 5
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 6
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 7
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 8
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 9
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 10
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 11
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
epoch: 12
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]